In [1]:
riot_api_key='RGAPI-4e35b70a-4214-485e-8ef3-0fa4e4d3a5b4'

In [2]:
# summoner_id와 riot_api_key를 가지고 puu_id를 가지고 온다.
def get_puu_id(summoner_id,riot_api_key):
    url = f'https://kr.api.riotgames.com/lol/summoner/v4/summoners/{summoner_id}?api_key={riot_api_key}'
    res = requests.get(url).json()
    puu_id = res['puuid']
    return puu_id

In [3]:
# puu_id와 page를 가지고 해당 puu_Id 사용자의 최근 매치게임을 가져온다. 
def get_match_ids(puu_id,riot_api_key,page=20):
    url = f'https://asia.api.riotgames.com/lol/match/v5/matches/by-puuid/{puu_id}/ids?type=ranked&start=0&count={page}&api_key={riot_api_key}'
    res = requests.get(url).json()
    return res

In [4]:
# match_ids를 가지고 매치랑 타임리스트를 받고, 
# match_info_list라는 리스트에 match_id와 res1(매치), res2(타임리스트), 티어를 저장한다. 
def get_matches_timelines(match_ids,riot_api_key, tier):
    match_info_list = []
    for match_id in tqdm(match_ids):
        url1 = f'https://asia.api.riotgames.com/lol/match/v5/matches/{match_id}?api_key={riot_api_key}'
        url2 = f'https://asia.api.riotgames.com/lol/match/v5/matches/{match_id}/timeline?api_key={riot_api_key}'
        res1 = requests.get(url1).json()
        res2 = requests.get(url2).json()

        if res1['info']['gameMode'] != 'CLASSIC' :
            print('예외')
            continue
        
        match_info_list.append([match_id,res1,res2, tier])
    return match_info_list

In [5]:
def get_champion_mastery(puuid, champion_id, riot_api_key):
    url = f'https://kr.api.riotgames.com/lol/champion-mastery/v4/champion-masteries/by-puuid/{puuid}/by-champion/{champion_id}?api_key={riot_api_key}'
    try:
        res = requests.get(url).json()
        return res.get('championPoints', 0)
    except requests.RequestException as e:
        print(f"{puuid}의 챔피언 {champion_id} 숙련도를 가져오는 중 오류 발생: {e}")
        return 0

In [6]:
import requests
import random
import pandas as pd
from tqdm import tqdm
import time
from my_utils import *

def get_rawdata(tier,riot_api_key):
    summoner_id_list = []
    puu_id_list = []
    divisions = ['I', 'II', 'III', 'IV']
    # 랜덤으로 페이지를 골라서 데이터를 출력한다. 
    #page = random.randrange(1,51)

    
    print(f'{tier} 소환사 ID 가져오는 중...')
    for division in divisions:
        try:
            url = f'https://kr.api.riotgames.com/lol/league/v4/grandmasterleagues/by-queue/RANKED_SOLO_5x5?api_key={riot_api_key}'
            res = requests.get(url).json()
            
            #summonerId_list에 summonerId 데이터를 담는다. 
            summonerId_all_list = []
            for entry in res['entries']:
                summonerId_all_list.append(entry['summonerId'])
            
            summoner_id_list=[]
            summoner_id_list.extend(random.sample(summonerId_all_list, 3))
        
        except requests.RequestException as e:
            print(f"{tier} {division} 소환사 목록을 가져오는 중 오류 발생: {e}")

    summoner_id_list = [x for x in summoner_id_list]
    print('PUUID 가져오는 중...')
    
    for summoner_id in tqdm(summoner_id_list, desc="PUUID 가져오는 중"):
        puu_id = get_puu_id(summoner_id, riot_api_key)
        if puu_id:
            puu_id_list.append(puu_id)

    print('경기 ID 가져오는 중...')
    match_id_list = []
    
    for puu_id in tqdm(puu_id_list, desc="경기 ID 가져오는 중"):
        match_ids = get_match_ids(puu_id, riot_api_key, 2)
        match_id_list.extend(match_ids)

    print('경기 정보 가져오는 중...')
    match_data = get_matches_timelines(match_id_list, riot_api_key, tier)
    df = pd.DataFrame(match_data, columns=['match_id', 'matches', 'timeline','tier'])

    df_info = []
    print('소환사 통계 생성 중...')
        
    for _, row in tqdm(df.iterrows(), desc="소환사 통계 생성 중", total=len(df)):
        match_info = row['matches']['info']
        if match_info['gameDuration'] > 780:
            for participant in match_info['participants']:
                #try:
                    tmp = [
                        row['match_id'],
                        match_info['gameDuration'],
                        match_info['gameVersion'],
                        participant['riotIdGameName'],
                        participant['riotIdTagline'],
                        participant['summonerLevel'],
                        participant['participantId'],
                        participant['championId'],
                        participant['championName'],
                        get_champion_mastery(participant['puuid'], participant['championId'], riot_api_key),
                        participant['teamPosition'],
                        participant['teamId'],
                        1 if participant['win'] else 0,
                        participant['kills'],
                        participant['deaths'],
                        participant['assists'],
                        participant['totalDamageDealtToChampions'],
                        participant['totalDamageDealt'],
                        row['tier']
                    ]
                    df_info.append(tmp)
                #except KeyError as e:
                    #print(f"데이터 처리 중 키 오류 발생: {e}")
                    continue

    columns = [
        'match_id', 'gameDuration', 'gameVersion', 'riotIdGameName', 'tagLine', 'summonerLevel', 
        'participantId', 'championId', 'championName', 'championMastery', 'teamPosition', 
        'teamId', 'win', 'kills', 'deaths', 'assists', 'totalDamageDealtToChampions', 
        'totalDamageDealt', 'tier'
    ]

    df_result = pd.DataFrame(df_info, columns=columns).drop_duplicates()
    print(f'{tier} 티어 데이터프레임의 행 수는 {len(df_result)}입니다.')

    # 데이터프레임을 리스트로 변환
    data_list = df_result.values.tolist()

    # 리스트 출력
    for data in data_list:
        print(data)

    return df_result

In [7]:
def insert_data(row):
    query = (
        f"INSERT INTO testMATCHS ("
        f"MCID, MCGDURATION, MCGVER, MCRDGANAME, MCRDTAGLINE, MCSUMMONERLEVEL, "
        f"MCPARTICIPANTID, MCCHAMPIONID, MCCHAMPIONNAME, MCCHAMPEXPERIENCE, MCTIEAMPOSITION, "
        f"MCTEAMID, MCWIN, MCKILLS, MCDEATHS, MCASSISTS, MCTOTALDDCHAMPIONS, "
        f"MCTOTALDTAKEN, MCTIER) "
        f"VALUES ("
        f"{repr(row.match_id)}, {row.gameDuration}, {repr(row.gameVersion)}, {repr(row.riotIdGameName)}, {repr(row.tagLine)}, {row.summonerLevel}, "
        f"{row.participantId}, {row.championId}, {repr(row.championName)}, {row.championMastery}, {repr(row.teamPosition)}, "
        f"{row.teamId}, {row.win}, {row.kills}, {row.deaths}, {row.assists}, {row.totalDamageDealtToChampions}, "
        f"{row.totalDamageDealt}, {repr(row.tier)})"
    )
    sql_execute(query)

In [8]:
tier = 'GRANDMASTER'
# tqdm 설정 추가
tqdm.pandas()

for _ in range(100000000000):
    for i in range(3):
        try:
            db_open()  # 데이터베이스 연결
            df = get_rawdata(tier, riot_api_key)
            df.progress_apply(lambda x: insert_data(x), axis=1) # insert 코드
            db_close()  # 데이터베이스 연결 닫기
            time.sleep(100)  # Riot API는 2분에 100개
        except Exception as e:
            print(f"{tier} 티어에서 오류 발생: {e}")
            #print("오류가 발생한 데이터:", df)  # 추가: 오류가 발생한 데이터 출력
            time.sleep(100)
            continue


GRANDMASTER 소환사 ID 가져오는 중...
PUUID 가져오는 중...


PUUID 가져오는 중: 100%|█████████████████████████████████████████████████████████████████| 3/3 [00:01<00:00,  2.46it/s]


경기 ID 가져오는 중...


경기 ID 가져오는 중: 100%|███████████████████████████████████████████████████████████████| 3/3 [00:01<00:00,  2.20it/s]


경기 정보 가져오는 중...


...

경기 정보 가져오는 중...


100%|████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:08<00:00,  1.33s/it]


소환사 통계 생성 중...


소환사 통계 생성 중:  83%|████████████████████████████████████████████████████▌          | 5/6 [00:29<00:05,  5.82s/it]


KeyboardInterrupt: 

In [ ]:
#db_open()

In [ ]:
#df = sql_execute('select * from testMATCHS')